In [50]:
import calcOneDay
import getDays
from datetime import datetime, timedelta
import calcTimeNow
import daysAndDates
import logging
import checkDST

def logdV3():
    
    """Set up logging for this script"""
    
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')
    #file_handler = logging.FileHandler('/home/ec2-user/davisPMUpdate.log')
    file_handler = logging.FileHandler('/Users/jameshayes/Sites/dV3Up.log')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)  

In [63]:
import getDays
from datetime import datetime, timedelta
import calcTimeNow
import daysAndDatesNew
import checkDST
from dataFromDavis import dataFromDavis
from getAndStore import getAndStore
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from dateutil.tz import tzutc, tzlocal
import pytz

def timesAndDates():
    
    """Calculate the time and date for end of day calculations
    First determine whether we are in DST or not"""

    # Check DST setting
    isDST = checkDST.checkDST()
    print(isDST)

    if isDST == 1:
        now = datetime.now()
        end = int(datetime.timestamp(now))
        start = (end - 55800)
        start = str(start)
        end = str(end)
        
    else:
        now = datetime.now()
        end = int(datetime.timestamp(now))
        start = (end - 59400)
        start = str(start)
        end = str(end)    

    dayInfo = daysAndDatesNew.daysAndDatesNew()
    month, month_num, date, year = dayInfo[0], dayInfo[1], dayInfo[2], dayInfo[3]
    yesterday = int(dayInfo[4])
    nextDay = int(dayInfo[5])
    month_num = int(month_num)
    date = int(date)
        
    return(start, end, month, month_num, date, year, yesterday, nextDay)  

timesAndDates()
dataFromDavis()
getAndStore()

1
('March', '03', '20', '2024')
March 03 21 2024
1710998504 1711054304


,timestamp,temp_hi,temp_hi_at,temp_lo,temp_lo_at,rainfall_in,dew_point_hi,dew_point_lo,rain_rate_hi_in,rain_rate_hi_at,timeGroup,localTime
0,1710965820,54.30,1710965775,54.20,1710965786,0,43.40,43.20,0,1710965770,2024-03-20 16:17:00-04:00,04:17 PM
1,1710965880,54.30,1710965838,54.30,1710965838,0,43.50,43.40,0,1710965823,2024-03-20 16:18:00-04:00,04:18 PM
2,1710965940,54.30,1710965891,54.20,1710965933,0,43.50,43.30,0,1710965886,2024-03-20 16:19:00-04:00,04:19 PM
3,1710966000,54.30,1710965954,54.20,1710965975,0,43.40,43.20,0,1710965949,2024-03-20 16:20:00-04:00,04:20 PM
4,1710966060,54.30,1710966017,54.20,1710966048,0,43.30,43.20,0,1710966001,2024-03-20 16:21:00-04:00,04:21 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
1435,1711051920,44.30,1711051878,44.20,1711051910,0,13.10,13.00,0,1711051863,2024-03-21 16:12:00-04:00,04:12 PM
1436,1711051980,44.10,1711051931,44.10,1711051931,0,12.90,12.90,0,1711051926,2024-03-21 16:13:00-04:00,04:13 PM
1437,1711052040,44.30,1711052015,44.20,1711051994,0,13.00,12.60,0,1711051989,2024-03-21 16:14:00-04:00,04:14 PM
1438,1711052100,44.20,1711052057,44.10,1711052099,0,12.60,11.90,0,1711052041,2024-03-21 16:15:00-04:00,04:15 PM


In [61]:
import pandas as pd
from pandas import DataFrame, Series
import math
from tableWrite import tableWrite
import daysAndDatesNew

def createF6(df):
    
    """Create FG data from the newly created and passed dataframe"""
    
    max_temp  = (df.sort_values(by='temp_hi', ascending=False))
    max_T = max_temp.iloc[:1]
    max_T_time = int(max_T['temp_hi_at'])
    maxT = max_T['temp_hi'].values[0]
    maxT = round(maxT)
    print(maxT)

    min_temp  = (df.sort_values(by='temp_lo', ascending=True))
    min_T = min_temp.iloc[:1]
    min_T_time = int(min_T['temp_lo_at'])
    minT = min_T['temp_lo'].values[0]
    minT = round(minT)

    dew_max = (df.sort_values(by='dew_point_hi', ascending=False))
    dew_max1 = dew_max.iloc[:1]
    dewMax = dew_max1['dew_point_hi'].values[0]
    dewMaxT = round(dewMax)

    dew_min = (df.sort_values(by='dew_point_lo', ascending=True))
    dew_min1 = dew_min.iloc[:1]
    dewMin = dew_min1['dew_point_lo'].values[0]
    dewMinT = round(dewMin)

    rain = df['rainfall_in'].sum()
    avgTemp = math.ceil((int(maxT + minT)/2))

    hdd = (65 - avgTemp)
    if hdd < 0:
        hdd = 0
    cdd = (avgTemp - 65)
    if cdd < 0:
        cdd = 0          
 
    return(maxT, minT, dewMaxT, dewMinT, rain, avgTemp, hdd, cdd)
    
df = getAndStore()
createF6(df)

60


(60, 29, 44, 9, 0, 45, 20, 0)

In [56]:
import sandbox1
import sandbox2
import daysAndDatesNew
import createF6

#
# Import data for use in the alamanc
#

dayInfo = daysAndDatesNew.daysAndDatesNew()
month, month_num, date, year = dayInfo[0], dayInfo[1], dayInfo[2], dayInfo[3]
yesterday = int(dayInfo[4])
nextDay = int(dayInfo[5])

sdf = createF6.createF6(df)
print(sdf)
maxT, minT, dewMaxT, throttle, rain, avgTemp, hdd, cdd = sdf[0], sdf[1], sdf[2], sdf[3], sdf[4], sdf[5], sdf[6], sdf[7]

nmlData = sandbox2.sandbox2()
nmlHi = nmlData[3]
nmlLo = nmlData[4]

highData = sandbox1.recordHigh()
lowData = sandbox1.recordLow()
rainData = sandbox1.recordRain()

highPhrase = highData[2]
lowPhrase = lowData[2]
rainPhrase = rainData[2]

#filePath = '/var/www/html/000/climoDavisTest111.txt'
filePath = '/Users/jameshayes/Sites/climoDavis999.txt'

def almanac():
    
    """Set and and write the daily almanac"""

    with open(f'{filePath}','w') as outfile1: 
    #with open('/var/www/html/000/climoDavisText.txt','w') as outfile1: 
        print(f'Daily almanac for {month} {date}, {year}', file = outfile1)
        print('\n', file = outfile1)
        print(f'The high so far today is {maxT} degrees', file = outfile1)
        print(f'The low so far today is {minT} degrees', file = outfile1)
        print(f'The average temperature is {avgTemp} degrees', file = outfile1)
        print(f'The rainfall so far today is {("%.2f" % rain)} inches', file = outfile1)
        if hdd == 0:
            print('')
        else:
            print(f'There were {hdd} heating degree days', file = outfile1)
        if cdd == 0:
            print('')
        else:
            print(f'There were {cdd} cooling degree days', file = outfile1)
            
        print('\n', file = outfile1)        
    
    
        if date == 1:
            print(f'Normal and Record information for {nextMonth} {nextDay}, {year}', file = outfile1)
            print('\n', file = outfile1)
            print(f'The normal high for today is {nmlHi} degrees', file = outfile1)
            print(f'The normal low for today is {nmlLo} degrees' , file = outfile1)
            print('\n', file = outfile1)
            print(highPhrase, file = outfile1)
            print(lowPhrase, file = outfile1)
            print(rainPhrase, file = outfile1)  
        
        else:
            print(f'Normal and Record information for {month} {date}, {year}', file = outfile1)
            print('\n', file = outfile1)
            print(f'The normal high for today is {nmlHi} degrees', file = outfile1)
            print(f'The normal low for today is {nmlLo} degrees' , file = outfile1)
            print('\n', file = outfile1)
            print(highPhrase, file = outfile1)
            print(lowPhrase, file = outfile1)
            print(rainPhrase, file = outfile1)  
            
almanac()            
        

('March', '03', '20', '2024')
(60, 29, 44, 9, 0, 45, 20, 0)
1
2001

